# Group Project: University Course Timetabling Problem (Base Model)

Course Title: **DAMO-610-3: Winter 2025 Operations Analytics**  
Professor Name: **Professor Hany Osman**  
Submission Date: **March 23, 2025**  
Submitted By:  
1. **Catherine C. Calantoc (NF1001422)**  
2. **Dev D. Rabadia (NF1005560)**  
3. **Miko L. Tan (NF1008647)**  
4. **Rosario D. Torres (NF1001385)**  

**Class Scheduling and Instructor Assignment in Higher Education:** In higher education, class scheduling and instructor assignment is a multi-faceted problem requiring the balancing of faculty availability, classroom capacity, student preferences, and the curriculum. Inefficient scheduling results in wasted resources, conflicts, and dissatisfaction among students and instructors. Integer programming optimization models enable universities to automate and optimize their timetables to schedule courses appropriately while balancing instructor workload, maximizing student accessibility, and minimizing operational cost. An optimal scheduling system provides the backbone for academic planning, reducing administrative hassle and improving the learning experience for students.

In [243]:
from ortools.sat.python import cp_model
import pandas as pd
import time

## Step 1: Define the Data Model

In [244]:
def create_base_data_model():
    """
        Creates and returns the data model for the University Course Timetabling Problem.
        Data used was based on UNFC Course Catalog for Sprint 2025.
    """

    # Load the Excel file
    file_path = "uctp-base-model-data.xlsx"

    try:
        dfs = pd.read_excel(file_path, sheet_name=None)
        print("Data loaded successfully from", file_path)
    except Exception as e:
        print(f"Error loading data from {file_path}: {e}")
        raise

    # Autofill missing Preference columns with 2 (default)
    for sheet in ["Lecturer_Course", "Lecturer_Day"]:
        if sheet in dfs and "Preference" in dfs[sheet].columns:
            dfs[sheet]["Preference"] = dfs[sheet]["Preference"].fillna(2).astype(int)
        # If the sheet exists but does not have a "Preference" column, create a new "Preference" column and assigns default value to all rows.
        elif sheet in dfs:
            dfs[sheet]["Preference"] = 2

    # Autofill missing Preference columns with 1 (default)
    for sheet in ["Course_Day", "Course_Room"]:
        if sheet in dfs and "Preference" in dfs[sheet].columns:
            dfs[sheet]["Preference"] = dfs[sheet]["Preference"].fillna(1).astype(int)
        # If the sheet exists but does not have a "Preference" column, create a new "Preference" column and assigns default value to all rows.
        elif sheet in dfs:
            dfs[sheet]["Preference"] = 1

    # Return all dataframes as a tuple
    return dfs

## Step 2: Define the Model

In [245]:
# Define the CP-SAT model - Constraint Programming (CP)
model = cp_model.CpModel()

## Step 3: Define Indices, Parameters, and Decision Variables

### **Indices and Parameters**
- $L$: The number of lecturers (professors)  
- $C$: The number of courses  
- $R$: The number of rooms (classrooms)  
- $D$: The number of schooldays (days)  per week
- $T$: The number of timeslots per day
- $l$: Index for lecturers where *l*={1,2,… ,*L*}  
- $c$: Index for courses where *c*={1,2,… ,*C*}  
- $r$: Index for rooms where *r*={1,2,… ,*R*}  
- $d$: Index for school days *d*={1,2,… ,*D*}  
- $t$: Index for timeslots where *t*={1,2,…,*T*}  
- $p1_{l,c}$: The preference of lecturer *l* for teaching course *c*.  
- $p2_{l,d}$: The preference of lecturer *l* for being invited on schoolday *d*.  
- $p3_{c,d}$: The preference of course *c* for being presented on schoolday *d*.  
- $a1_{l,d}$: Parameter taking value 1 if lecturer *l* can be invited on schoolday *d*, and 0 otherwise.  
- $a2_{l,c}$: Parameter taking value 1 if lecturer *l* can teach course *c*, and 0 otherwise.
- $a3_{c,r}$: Parameter taking value 1 if course *c* can be presented in classroom *r*, and 0 otherwise.

In [246]:
dfs = create_base_data_model()

# Indices
L, C, R, D, T = len(dfs["Lecturer"]), len(dfs["Course"]), len(dfs["Room"]), len(dfs["Day"]), len(dfs["Time Period"])

# Parameters (Soft Constraints)
p1 = {(row["Lecturer Name"], row["Course Code"]): 1 if row["Preference"] >= 1 else 0 for _, row in dfs["Lecturer_Course"].iterrows()}
p2 = {(row["Lecturer Name"], row["Day"]): 1 if row["Preference"] >= 1 else 0 for _, row in dfs["Lecturer_Day"].iterrows()}
p3 = {(row["Course Code"], row["Day"]): 1 if row["Preference"] == 1 else 0 for _, row in dfs["Course_Day"].iterrows()}

# Parameters (Hard Constraints)
a1 = {(row["Lecturer Name"], row["Day"]): row["Preference"] for _, row in dfs["Lecturer_Day"].iterrows()}  # 0, 1, or 2
a2 = {(row["Lecturer Name"], row["Course Code"]): row["Preference"] for _, row in dfs["Lecturer_Course"].iterrows()}  # 0, 1, or 2
a3 = {(row["Course Code"], row["Room"]): 1 if row["Preference"] == 1 else 0 for _, row in dfs["Course_Room"].iterrows()}  # 0 or 1

# For data feasibility checks
is_feasible = True

Data loaded successfully from uctp-base-model-data.xlsx


### **Decision Variables**

$$Z_{l,c,d,r,t} =
\begin{cases} 
1 & \text{if lecturer } l \text{ teaches course } c \text{ on school day per week } d \text{ in room } r \text{ during timeslot per day } t, \\
0 & \text{otherwise.}
\end{cases}$$

In [247]:
# Z_{l,c,d,r,t}: 1 if lecturer l teaches course c on day d in room r at timeslot t, 0 otherwise
Z = {(l, c, d, r, t): model.NewBoolVar(f'Z_{l}_{c}_{d}_{r}_{t}')
     for l in dfs["Lecturer"]["Lecturer Name"] for c in dfs["Course"]["Course Code"] for d in dfs["Day"]["Day"]
     for r in dfs["Room"]["Room Number"] for t in dfs["Time Period"]["Time Period"]}

$$X_{l,d} =
\begin{cases} 
1 & \text{if lecturer } l \text{ is scheduled to teach on school day per week } d, \\
0 & \text{otherwise.}
\end{cases}$$

In [248]:
# X_{l,d}: 1 if lecturer l is scheduled on day d, 0 otherwise
X = {(l, d): model.NewBoolVar(f'X_{l}_{d}')
     for l in dfs["Lecturer"]["Lecturer Name"] for d in dfs["Day"]["Day"]}

## Step 4: Define the Constraints

### **Constraint 1: One Schedule per Course**

**Purpose:** Ensures each course is scheduled exactly once, assigning it to one lecturer, day, room, and timeslot combination.

##### **Formula:**
$$ \sum_{l=1}^{L} \sum_{d=1}^{D} \sum_{r=1}^{R} \sum_{t=1}^{T} Z_{l,c,d,r,t} =1, \quad \forall c $$

In [249]:
# One schedule per course
# Data Feasibility Check - BEGIN
print("Checking Constraint #1: One Schedule per Course:")
if not dfs["Course"]["Course Code"].empty:
    for c in dfs["Course"]["Course Code"]:
        available_lecturers = [l for l in dfs["Lecturer"]["Lecturer Name"] if a2.get((l, c), 0) >= 1]
        available_rooms = [r for r in dfs["Room"]["Room Number"] if a3.get((c, r), 0) == 1]
        if not available_lecturers:
            is_feasible = False
            print(f"ERROR - Course {c} has no available lecturers (no a1 entries with Preference >= 1)")
        if not available_rooms:
            is_feasible = False
            print(f"ERROR - Course {c} has no suitable rooms (no a3 entries with Preference = 1)")
        if available_lecturers and available_rooms:
            print(f"OK - Course {c}: {len(available_lecturers)} lecturers, {len(available_rooms)} rooms available")
else:
    is_feasible = False
    print("ERROR - No courses found in Course sheet (Course Code column is empty)")
    print("Error - No courses to schedule")
# Data Feasibility Check - END

for c in dfs["Course"]["Course Code"]:
    model.AddExactlyOne(Z[l, c, d, r, t]
                        for l in dfs["Lecturer"]["Lecturer Name"] for d in dfs["Day"]["Day"]
                        for r in dfs["Room"]["Room Number"] for t in dfs["Time Period"]["Time Period"])

Checking Constraint #1: One Schedule per Course:
OK - Course DAMO-500: 15 lecturers, 9 rooms available
OK - Course DAMO-501: 14 lecturers, 9 rooms available
OK - Course DAMO-510: 15 lecturers, 9 rooms available
OK - Course DAMO-511: 15 lecturers, 9 rooms available
OK - Course DAMO-600: 15 lecturers, 9 rooms available
OK - Course DAMO-610: 15 lecturers, 9 rooms available
OK - Course DAMO-611: 15 lecturers, 9 rooms available
OK - Course DAMO-621: 15 lecturers, 9 rooms available
OK - Course DAMO-622: 15 lecturers, 9 rooms available
OK - Course DAMO-623: 15 lecturers, 9 rooms available
OK - Course CPSC-500: 15 lecturers, 9 rooms available
OK - Course CPSC-510: 15 lecturers, 9 rooms available
OK - Course CPSC-600: 15 lecturers, 9 rooms available
OK - Course CPSC-610: 15 lecturers, 9 rooms available
OK - Course CPSC-620: 15 lecturers, 9 rooms available


### **Constraint 2: No Lecturer Overlap**

**Purpose:** Prevents a lecturer from being scheduled for multiple courses in the same timeslot on a given day (no overlap).​

##### **Formula:**
$$ \sum_{c=1}^{C} \sum_{r=1}^{R} Z_{l,c,d,r,t} \leq X_{l,d} \quad \forall l,d,t $$

In [250]:
# No lecturer overlap
# Data Feasibility Check - BEGIN
print("Checking Constraint #2: No Lecturer Overlap:")
total_slots = D * R * T
if total_slots < C:
    is_feasible = False
    print(f"ERROR - Insufficient slots: {C} courses require at least {C} slots, but only {total_slots} available (D={D}, R={R}, T={T})")
else:
    print(f"OK - Total slots ({total_slots}) sufficient for {C} courses")
# Data Feasibility Check - END

for l in dfs["Lecturer"]["Lecturer Name"]:
    for d in dfs["Day"]["Day"]:
        # Link X_{l,d} to Z: X_{l,d} = 1 if any Z_{l,c,d,r,t} = 1
        model.Add(sum(Z[l, c, d, r, t] for c in dfs["Course"]["Course Code"]
                      for r in dfs["Room"]["Room Number"] for t in dfs["Time Period"]["Time Period"]) <= X[l, d] * C)
        # No overlap: At most one course per timeslot
        for t in dfs["Time Period"]["Time Period"]:
            model.AddAtMostOne(Z[l, c, d, r, t]
                               for c in dfs["Course"]["Course Code"] for r in dfs["Room"]["Room Number"])

Checking Constraint #2: No Lecturer Overlap:
OK - Total slots (135) sufficient for 15 courses


### **Constraint 3: Lecturer-Day Feasibility**

**Purpose:** Ensures a lecturer is only marked as scheduled on a day if they are available on that day, enforcing lecturer-day preferences or availability.

##### **Formula:**
$$ X_{l,d} \leq a1_{l,d}, \quad \forall l,d $$

In [251]:
# Lecturer-day feasibility
# Data Feasibility Check - BEGIN
print("Checking Constraint #3: Lecturer-Day Feasibility:")
for l in dfs["Lecturer"]["Lecturer Name"]:
    available_days = [d for d in dfs["Day"]["Day"] if a1.get((l, d), 0) >= 1]
    if not available_days:
        is_feasible = False
        print(f"ERROR - Lecturer {l} has no available days (no a2 entries with Preference >= 1)")
    else:
        print(f"OK - Lecturer {l}: {len(available_days)} available days")
# Data Feasibility Check - END

for l in dfs["Lecturer"]["Lecturer Name"]:
    for d in dfs["Day"]["Day"]:
        model.Add(X[l, d] <= a1.get((l, d), 0))

Checking Constraint #3: Lecturer-Day Feasibility:
OK - Lecturer Abbas Hamze: 5 available days
OK - Lecturer Ahmed Eltahawi: 5 available days
OK - Lecturer Ali El-Sharif: 5 available days
OK - Lecturer Cosimo Girolamo: 5 available days
OK - Lecturer Hany Osman: 5 available days
OK - Lecturer Hassan Baz Chamas: 5 available days
OK - Lecturer Marin Vratonjic: 5 available days
OK - Lecturer Mehdi Mostofi: 5 available days
OK - Lecturer Mohannad Al Mousa: 5 available days
OK - Lecturer Omar Nada: 5 available days
OK - Lecturer Omid Isfahanialamdari: 5 available days
OK - Lecturer Sundus Shanef: 5 available days
OK - Lecturer Touraj Banirostam: 5 available days
OK - Lecturer Yiran Hao: 5 available days
OK - Lecturer Zeeshan Ahmad: 5 available days


### **Constraint 4: No Room Overlap**

**Purpose:** Prevents a room from being scheduled for multiple courses at the same time, ensuring exclusive use of each room per day and timeslot.

##### **Formula:**
$$ \sum_{l=1}^{L} \sum_{c=1}^{C} Z_{l,c,d,r,t} \leq 1, \quad \forall d,r,t $$

In [252]:
# No room overlap
# Data Feasibility Check - BEGIN
print("Checking Constraint #4: No Room Overlap:")
print(f"OK - Room slots ({total_slots}) checked against {C} courses")
# Data Feasibility Check - END

for r in dfs["Room"]["Room Number"]:
    for d in dfs["Day"]["Day"]:
        for t in dfs["Time Period"]["Time Period"]:
            model.AddAtMostOne(Z[l, c, d, r, t]
                               for l in dfs["Lecturer"]["Lecturer Name"] for c in dfs["Course"]["Course Code"])

Checking Constraint #4: No Room Overlap:
OK - Room slots (135) checked against 15 courses


### **Constraint 5: Lecturer-Course Feasibility**

**Purpose:** Ensures a lecturer can only teach a course if they are qualified or assigned to it, limiting total assignments of that course across all days, rooms, and timeslots (typically to 0 or 1 due to Constraint 1).

##### **Formula:**
$$ \sum_{d=1}^{D} \sum_{r=1}^{R} \sum_{t=1}^{3} Z_{l,c,d,r,t} \leq a2_{l,c}, \quad \forall l,c $$

In [253]:
# Lecturer-course feasibility
# Data Feasibility Check - BEGIN
print("Checking Constraint #5: Lecturer-Course Feasibility:")
for l in dfs["Lecturer"]["Lecturer Name"]:
    assigned_courses = [c for c in dfs["Course"]["Course Code"] if a2.get((l, c), 0) >= 1]
    available_slots = sum(a1.get((l, d), 0) for d in dfs["Day"]["Day"]) * len(dfs["Time Period"]["Time Period"])
    if available_slots < len(assigned_courses):  # Compare total available slots to courses
        is_feasible = False
        print(f"ERROR - Lecturer {l} has {len(assigned_courses)} courses but only {available_slots} available slots")
    else:
        print(f"OK - Lecturer {l}: {len(assigned_courses)} courses, {available_slots} available slots")
# Data Feasibility Check - END

for l in dfs["Lecturer"]["Lecturer Name"]:
    for c in dfs["Course"]["Course Code"]:
        model.Add(sum(Z[l, c, d, r, t] for d in dfs["Day"]["Day"]
                      for r in dfs["Room"]["Room Number"] for t in dfs["Time Period"]["Time Period"]) <= a2.get((l, c), 0))

Checking Constraint #5: Lecturer-Course Feasibility:
OK - Lecturer Abbas Hamze: 14 courses, 30 available slots
OK - Lecturer Ahmed Eltahawi: 15 courses, 30 available slots
OK - Lecturer Ali El-Sharif: 15 courses, 30 available slots
OK - Lecturer Cosimo Girolamo: 15 courses, 30 available slots
OK - Lecturer Hany Osman: 15 courses, 30 available slots
OK - Lecturer Hassan Baz Chamas: 15 courses, 30 available slots
OK - Lecturer Marin Vratonjic: 15 courses, 30 available slots
OK - Lecturer Mehdi Mostofi: 15 courses, 30 available slots
OK - Lecturer Mohannad Al Mousa: 15 courses, 30 available slots
OK - Lecturer Omar Nada: 15 courses, 30 available slots
OK - Lecturer Omid Isfahanialamdari: 15 courses, 30 available slots
OK - Lecturer Sundus Shanef: 15 courses, 30 available slots
OK - Lecturer Touraj Banirostam: 15 courses, 30 available slots
OK - Lecturer Yiran Hao: 15 courses, 30 available slots
OK - Lecturer Zeeshan Ahmad: 15 courses, 30 available slots


### **Constraint 6: Course-Room Fit**

**Purpose:** Ensures a course can only be scheduled in a room if the room is suitable limiting assignments to compatible rooms based on preferences or requirements.

##### **Formula:**
$$ \sum_{l=1}^{L} \sum_{d=1}^{D} \sum_{t=1}^{3} Z_{l,c,d,r,t} \leq a3_{c,r}, \quad \forall c,r $$

In [254]:
# Course-room fit
# Data Feasibility Check - BEGIN
print("\nChecking Constraint 6: Course-room fit")
for c in dfs["Course"]["Course Code"]:
    rooms = [r for r in dfs["Room"]["Room Number"] if a3.get((c, r), 0) == 1]
    print(f"OK - Course {c}: {len(rooms)} suitable rooms (checked in Constraint 1)")
# Data Feasibility Check - END

for c in dfs["Course"]["Course Code"]:
    for r in dfs["Room"]["Room Number"]:
        model.Add(sum(Z[l, c, d, r, t] for l in dfs["Lecturer"]["Lecturer Name"]
                      for d in dfs["Day"]["Day"] for t in dfs["Time Period"]["Time Period"]) <= a3.get((c, r), 0))


Checking Constraint 6: Course-room fit
OK - Course DAMO-500: 9 suitable rooms (checked in Constraint 1)
OK - Course DAMO-501: 9 suitable rooms (checked in Constraint 1)
OK - Course DAMO-510: 9 suitable rooms (checked in Constraint 1)
OK - Course DAMO-511: 9 suitable rooms (checked in Constraint 1)
OK - Course DAMO-600: 9 suitable rooms (checked in Constraint 1)
OK - Course DAMO-610: 9 suitable rooms (checked in Constraint 1)
OK - Course DAMO-611: 9 suitable rooms (checked in Constraint 1)
OK - Course DAMO-621: 9 suitable rooms (checked in Constraint 1)
OK - Course DAMO-622: 9 suitable rooms (checked in Constraint 1)
OK - Course DAMO-623: 9 suitable rooms (checked in Constraint 1)
OK - Course CPSC-500: 9 suitable rooms (checked in Constraint 1)
OK - Course CPSC-510: 9 suitable rooms (checked in Constraint 1)
OK - Course CPSC-600: 9 suitable rooms (checked in Constraint 1)
OK - Course CPSC-610: 9 suitable rooms (checked in Constraint 1)
OK - Course CPSC-620: 9 suitable rooms (checked in

### **Constraint 7: Binary Constraint for Lecturer Assignment**

**Purpose:** A lecturer assignment is a binary decision.

##### **Formula:**
$$ X_{l,d} \in \{0,1\}, \quad \forall l,d $$

In [255]:
# Constraint - X is Binary (Already Enforced by Definition)
# No additional constraint is needed since X is defined as a binary variable.

### **Constraint 8: Binary Constraint for Course Assignment**

**Purpose:** A course assignment is a binary decision.

##### **Formula:**
$$ Z_{l,c,d,r,t} \in \{0,1\}, \quad \forall l,c,d,r,t $$

In [256]:
# Constraint - Z is Binary (Verification)
# No additional constraint is needed since Z[l, c, d, r, t] is already defined as a binary variable.
# No action needed; this is just to confirm the property

## Step 5: Define the Objective

##### $$ \text{Max } Z = \sum_{l=1}^{L} \sum_{d=1}^{D} X_{l,d} p2_{l,d} + \sum_{l=1}^{L} \sum_{c=1}^{C} \sum_{d=1}^{D} \sum_{r=1}^{R} \sum_{t=1}^{T} Z_{l,c,d,r,t} p1_{l,c} + \sum_{l=1}^{L} \sum_{c=1}^{C} \sum_{d=1}^{D} \sum_{r=1}^{R} \sum_{t=1}^{T} Z_{l,c,d,r,t} p3_{c,d} $$

In [257]:
# Objective Function
model.Maximize(
        sum(X[l, d] * p2[l, d] for l in dfs["Lecturer"]["Lecturer Name"] for d in dfs["Day"]["Day"]) +
        sum(Z[l, c, d, r, t] * p1[l, c]
            for l in dfs["Lecturer"]["Lecturer Name"] for c in dfs["Course"]["Course Code"]
            for d in dfs["Day"]["Day"] for r in dfs["Room"]["Room Number"] for t in dfs["Time Period"]["Time Period"]) +
        sum(Z[l, c, d, r, t] * p3[c, d]
            for l in dfs["Lecturer"]["Lecturer Name"] for c in dfs["Course"]["Course Code"]
            for d in dfs["Day"]["Day"] for r in dfs["Room"]["Room Number"] for t in dfs["Time Period"]["Time Period"])
    )

## Step 6: Solve the Model

In [258]:
# Solve the Model
solver = cp_model.CpSolver()
start_time = time.time()
status = solver.Solve(model)
solve_time = time.time() - start_time

## Step 7: Validate and Display the Results

In [259]:
# Check solution status and display results
output_file = "uctp-base-model-output.xlsx"

if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print(f"CP-SAT Objective Value: {solver.ObjectiveValue()}")
    print(f"CP-SAT Solve Time: {solve_time:.2f} seconds")
    
    assignments = [
        {'Lecturer': l, 'Course': c, 'Day': d, 'Room': r, 'Timeslot': t}
        for l, c, d, r, t in Z if solver.Value(Z[l, c, d, r, t]) > 0.5
    ]
    timetable_df = pd.DataFrame(assignments)
    print("\nCP-SAT Timetable:")
    print(timetable_df)
    
    timetable_df.to_excel(output_file, sheet_name="Timetable", index=False)
    print(f"\nTimetable saved to {output_file}")
else:
    print(f"CP-SAT failed to find a solution. Status: {solver.StatusName()}")
    print(f"Solve Time: {solve_time:.2f} seconds")
    
    diagnostic_data = {
        "Status": [solver.StatusName()],
        "Solve Time (s)": [solve_time],
        "Conflicts": [solver.NumConflicts()],
        "Branches": [solver.NumBranches()],
        "Wall Time (s)": [solver.WallTime()]
    }
    diagnostic_df = pd.DataFrame(diagnostic_data)
    diagnostic_df.to_excel(output_file, sheet_name="Diagnostics", index=False)
    print(f"\nDiagnostic information saved to {output_file}")

CP-SAT Objective Value: 105.0
CP-SAT Solve Time: 8.86 seconds

CP-SAT Timetable:
             Lecturer    Course        Day     Room     Timeslot
0         Abbas Hamze  DAMO-500     Monday  UNF 225  13:00-15:00
1         Abbas Hamze  DAMO-511     Monday  UNF 215  08:00-10:00
2         Abbas Hamze  DAMO-600  Wednesday  UNF 235  13:00-15:00
3         Abbas Hamze  DAMO-610    Tuesday  UNF 235  10:00-12:00
4         Abbas Hamze  DAMO-611   Thursday  UNF 403  10:00-12:00
5         Abbas Hamze  DAMO-621   Thursday  UNF 235  13:00-15:00
6         Abbas Hamze  DAMO-622    Tuesday  UNF 230  13:00-15:00
7         Abbas Hamze  DAMO-623   Thursday  UNF 235  08:00-10:00
8         Abbas Hamze  CPSC-500     Friday  UNF 240  10:00-12:00
9         Abbas Hamze  CPSC-510     Friday  UNF 403  08:00-10:00
10        Abbas Hamze  CPSC-600  Wednesday  UNF 235  10:00-12:00
11        Abbas Hamze  CPSC-610     Friday  UNF 245  13:00-15:00
12     Ahmed Eltahawi  DAMO-510    Tuesday  UNF 215  08:00-10:00
13  Mohan